In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [ ]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [ ]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()

        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # INPUT 28*28 | OUTPUT 26*26 | RF 3

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=8, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(8),
            nn.ReLU()
        ) # INPUT 26*26 | OUTPUT 24*24 | RF 5

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(8),
            nn.ReLU()
        ) # INPUT 24*24 | OUTPUT 22*22 | RF 7

        self.pool1 = nn.MaxPool2d(2, 2) # INPUT 22*22 | OUTPUT 11*11 | RF 8

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=20, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(20),
            nn.ReLU()
        ) # INPUT 11*11 | OUTPUT 9*9 | RF 12

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=16, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU()
        ) # INPUT 9*9 | OUTPUT 7*7 | RF 16


        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # INPUT 7*7 | OUTPUT 5*5 | RF 20

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # INPUT 5*5 | OUTPUT 3*3 | RF 24

        self.gap = nn.Sequential(
            nn.AvgPool2d(3)
        ) # INPUT 3*3 | OUTPUT 1*1 | RF 28

        self.dropout = nn.Dropout(0.01) #added dropout


    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.dropout(x)  #dropout
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.dropout(x) #dropout
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model2().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4            [-1, 8, 24, 24]             728
       BatchNorm2d-5            [-1, 8, 24, 24]              16
              ReLU-6            [-1, 8, 24, 24]               0
           Dropout-7            [-1, 8, 24, 24]               0
            Conv2d-8            [-1, 8, 22, 22]             584
       BatchNorm2d-9            [-1, 8, 22, 22]              16
             ReLU-10            [-1, 8, 22, 22]               0
        MaxPool2d-11            [-1, 8, 11, 11]               0
           Conv2d-12             [-1, 20, 9, 9]           1,460
      BatchNorm2d-13             [-1, 20, 9, 9]              40
             ReLU-14             [

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model =  Model2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.12093589454889297 Batch_id=468 Accuracy=94.22: 100%|██████████| 469/469 [00:16<00:00, 28.40it/s]



Test set: Average loss: 0.0962, Accuracy: 9843/10000 (98.43%)

EPOCH: 1


Loss=0.11352702230215073 Batch_id=468 Accuracy=98.36: 100%|██████████| 469/469 [00:16<00:00, 28.77it/s]



Test set: Average loss: 0.0601, Accuracy: 9882/10000 (98.82%)

EPOCH: 2


...


Test set: Average loss: 0.0243, Accuracy: 9927/10000 (99.27%)

EPOCH: 13


Loss=0.052587468177080154 Batch_id=468 Accuracy=99.42: 100%|██████████| 469/469 [00:16<00:00, 28.52it/s]



Test set: Average loss: 0.0306, Accuracy: 9912/10000 (99.12%)

EPOCH: 14


Loss=0.010730399750173092 Batch_id=468 Accuracy=99.40: 100%|██████████| 469/469 [00:16<00:00, 28.98it/s]



Test set: Average loss: 0.0251, Accuracy: 9935/10000 (99.35%)



**TARGET**

Keep the parameters less than 8000

Reduce the overfitting

Reach test accuracy > 99.4

**RESULT**

Parameters : 8,292

Best Training Accuracy : 99.42

Best Test Accuracy : 99.38

**ANALYSIS**

Parameters count is still greater than 8000

We could see model overfitting from 12th epoch

Even if we continue for few more epochs the model will not reach test accuracy > 99.4

**WHAT CAN BE DONE**

Reduce the parameter count by reduce the number of feature maps in intermediate layers

Add image augmentation

Increase the drop out